In [32]:
# 1.导入库和模块

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.ensemble import AdaBoostRegressor as ABR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge as R
from sklearn.linear_model import Lasso as L

In [33]:
# 2.查看数据

data = pd.read_csv('./dadas/MovieBoxOffiice.csv')
data.head()

,电影名,导演,编剧,演员,类型,电影时长(分钟),上映时间(年),评分,地区,语言,票房,发行公司,制片公司
0,角斗士 Gladiator,雷德利·斯科特,"大卫·弗兰佐尼,约翰·洛根,威廉姆·尼克尔森","罗素·克劳,华金·菲尼克斯,康妮·尼尔森,奥列佛·里德,理查德·哈里斯,德里克·雅各比,杰曼...",剧情,155,2000,8.5,德国,西班牙语,0.0,暂无发行公司,暂无制片公司
1,外出就餐3：饕餮自助 Eating Out 3: All You Can Eat,葛伦·盖洛,"菲利普·J·巴特尔,亚伦·布洛卡","利百加·科汉,克里斯·萨尔瓦多,迈克尔E.R.沃克,敏科·斯荳,莱斯利·乔丹,素玛立·蒙塔诺...",喜剧,80,2009,6.6,法国,汉语普通话,0.0,暂无发行公司,暂无制片公司
2,蝴蝶效应2 The Butterfly Effect 2,约翰·R·莱昂耐迪,Michael D. Weiss,"埃里克·里夫利,埃莉卡·杜兰斯,达斯汀·米利甘,吉娜·赫尔顿,林赛·麦克斯维尔",剧情,92,2006,6.1,印度,汉语粤语,0.0,暂无发行公司,暂无制片公司
3,铁血柔情 Love Me Tender,罗伯特·D·韦布,"莫里斯·杰拉蒂,罗伯特·巴克纳","Richard Egan,Debra Paget,Elvis Presley",剧情,89,1956,7.2,法国,德语,0.0,暂无发行公司,暂无制片公司
4,冰川时代 Ice Age,卡洛斯·沙尔丹哈,"迈克尔·伯格 ,迈克尔·J·威尔森,彼得·阿克曼","雷·罗马诺,约翰·雷吉扎莫,丹尼斯·利瑞,杰克·布莱克",喜剧,81,2002,8.6,英国,法语,0.0,暂无发行公司,暂无制片公司


In [34]:
# 3.选取分析数据

df = data.iloc[:,4:-2]
df.head()

,类型,电影时长(分钟),上映时间(年),评分,地区,语言,票房
0,剧情,155,2000,8.5,德国,西班牙语,0.0
1,喜剧,80,2009,6.6,法国,汉语普通话,0.0
2,剧情,92,2006,6.1,印度,汉语粤语,0.0
3,剧情,89,1956,7.2,法国,德语,0.0
4,喜剧,81,2002,8.6,英国,法语,0.0


In [39]:
# 4.建立预测票房函数

def predict_boxoffiice(df,movietype,movietime,movieyear,moviegraden,movieplace,movielanguage,movieboxoffiice,model):
    df.loc[len(df)] = [movietype,movietime,movieyear,moviegraden,movieplace,movielanguage,movieboxoffiice]
    
    dftype = pd.get_dummies(df['类型'],prefix='类型')
    datas = pd.concat([df,dftype],axis=1)
    dfplace = pd.get_dummies(df['地区'],prefix='地区')
    datas = pd.concat([datas,dfplace],axis=1)
    dflanguage = pd.get_dummies(df['语言'],prefix='语言')
    datas = pd.concat([datas,dflanguage],axis=1)
    
    datas = datas.loc[:,(datas.columns != '类型') & (datas.columns != '地区') & (datas.columns != '语言')]
    
    x = datas.loc[:,datas.columns != '票房'].values
    y = datas.loc[:,datas.columns == '票房'].values.reshape(-1,1)
    x_train, y_train, x_test, y_test = x[:-1], y[:-1], x[-1], y[-1]
    x_test = x_test.reshape(1,-1)
    
    Model = model()
    Model.fit(x_train,y_train)
    y_pred = Model.predict(x_test)
    
    return y_pred,datas

In [40]:
# 5.建立票房模型评价score函数

def predict_score(datas,model):
    x = datas.loc[:,datas.columns != '票房'].values
    y = datas.loc[:,datas.columns == '票房'].values.reshape(-1,1)
    x_train, x_test, y_train, y_test = model_selection.train_test_split(x,y,test_size=0.3)
    
    Model = model()
    Model.fit(x_train,y_train)
    score = Model.score(x_test,y_test)
    
    return score

In [43]:
# 6.打印各模型准确率即预测值

modellist = [MLP,ABR,DTR,RFR,LR,R,L]
for m in modellist:

    # movietype = input()
    # movietime = eval(input())
    # movieyear = eval(input())
    # moviegraden = eval(input())
    # movieplace = input()
    # movielanguage = input()
    # movieboxoffiice = eval(input())
    # model = input()

    movietype = '喜剧'
    movietime = 80
    movieyear = 2009
    moviegraden = 6.6
    movieplace = '法国'
    movielanguage = '汉语普通话'
    movieboxoffiice = 50000
    model = m
    
    boxoffiice,datas = predict_boxoffiice(df,movietype,movietime,movieyear,moviegraden,movieplace,movielanguage,movieboxoffiice,model)
    score = predict_score(datas,model)
    
    print(str(model)+'  '+str(score))
    
    boxoffiice_train = movieboxoffiice
    boxoffiice_test = "{:.2}".format(boxoffiice.flatten()[0])
    print('期望票房' + ' ' + str(boxoffiice_train))
    print('预测票房' + ' ' + str(boxoffiice_test))

D:\ANACONDA\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
D:\ANACONDA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\ANACONDA\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
D:\ANACONDA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
D:\ANACONDA\lib\site-packages\sklearn\ut

<class 'sklearn.neural_network._multilayer_perceptron.MLPRegressor'>  0.00043821783804054437
期望票房 50000
预测票房 2.5e+03


D:\ANACONDA\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


<class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>  -3.420621417645803
期望票房 50000
预测票房 1.7e+04
<class 'sklearn.tree._classes.DecisionTreeRegressor'>  -1.2070759014372343
期望票房 50000
预测票房 4e+04


D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


<class 'sklearn.ensemble._forest.RandomForestRegressor'>  -0.03255907606398978
期望票房 50000
预测票房 4.4e+04
<class 'sklearn.linear_model._base.LinearRegression'>  0.03239991617591054
期望票房 50000
预测票房 8.5e+03
<class 'sklearn.linear_model._ridge.Ridge'>  0.016689808633070347
期望票房 50000
预测票房 8.6e+03
<class 'sklearn.linear_model._coordinate_descent.Lasso'>  0.025703404155649978
期望票房 50000
预测票房 8.8e+03
